# Objective function `the best place for the company to grow`

In [1]:
# Objective function ->

# X (var independient): Val_by_place Each place will be assigned with a value in each component (9 x 1)

Val_by_place={"other Design companies":0,
         "kindergarden":0,
         "success startup":0,
         "Starbucks":0,
         "Airport_or_train_station":0,
         "place_to party":0,
         "basqueball court":0,
         "no_2km_old_company":0,
         "vegan_restaurant":0}

# Constants:
# Total personal (9 x 1)
Personal={"Designers":20,
         "UI/UX Engineers":5,
         "Frontend Developers":10,
         "Data Engineers":15,
         "Backend Developers":15,
         "Account Managers":20,
         "Maintenance guy":1,
         "Executives":10,
         "CEO/President":1}

# Valoration of the contribution to grow by role (9 x 1)
Val_to_grow={"Designers":5,
         "UI/UX Engineers":5,
         "Frontend Developers":5,
         "Data Engineers":5,
         "Backend Developers":5,
         "Account Managers":7,
         "Maintenance guy":3,
         "Executives":8,
         "CEO/President":10}

# Rel_personal_Val_place represent the relation between condition and personal (9 x 9)

Rel_personal_Val_place =[
    [1,0,0,0,0,0,0,0,0], # other Design companies -> Designers
    [0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3], # kindergarden -> 30% of the personal
    [0,1,1,1,1,0,0,0,0], # success startup
    [0,0,0,0,0,0,0,1,0], # Starbucks
    [0,0,0,0,0,1,0,0,0], # airport..
    [1,1,1,1,1,1,1,1,1], # place to party
    [0,0,0,0,0,0,1,0,0], # Basquetball court   
    [1,1,1,1,1,1,1,1,1], # No old company (10 years)
    [0,0,0,0,0,0,0,0,1], # vegan restauran
]

# Any places will be valuated with the next equation:
#               constant                              Independient Variable
# F = (Personal * Val_to_grow * Rel_personal_Val_place) * Val_by_place

### Other design companies

In [2]:
import json
from pymongo import MongoClient
url_mongo="mongodb://localhost:27017"
client = MongoClient(url_mongo)
db = client.get_database("companies")
result=list(db.companies.find({},{"category_code":1,"_id":0}))
categories=[e['category_code'] for e in result]

In [3]:
import pandas as pd
import numpy as np
categorias= pd.DataFrame(categories)
categorias=categorias.rename(columns={0:"categoria"})
len(categorias.categoria.value_counts())# -->  41 categories
len(categorias.categoria[categorias.categoria=="design"]) # just 4 companies!!
len(categorias.categoria[categorias.categoria=="web"]) # 3787.. is web equivalent to design?

3787

In [4]:
query={"$or":[{"category_code":"design"},{"category_code":"web"}]}
offices=list(db.companies.find(query,{"name":1,"offices":1,"_id":0}))
offices=pd.DataFrame(offices)
offices=offices.explode("offices")

In [5]:
def json_column(row):
    of=row.offices
    if type(of)==float or of["latitude"] is None:
        return ("theres is no data ",None, None)
    return ("success",of["latitude"] ,of["longitude"])

offices_coor=offices.apply(json_column, axis=1, result_type='expand')
print(type(offices))
offices_coor_name = pd.concat([offices.name,offices_coor])

<class 'pandas.core.frame.DataFrame'>


In [6]:
offices_coor_name=offices_coor_name.rename(columns={0:'State_coor',1:'latitude',2:'longitud'} )
offices_coor_name=offices_coor_name[offices_coor_name['State_coor']=="success"] # 2331 compnies with coordenates


In [14]:
def coor_jgeo(row):
     return {'type':'Point','coordinates':[row['longitud'],row['latitude']]}

offices_coor_name['design_coor']=offices_coor_name.apply(coor_jgeo,axis=1)

In [15]:
offices_coor_name

,State_coor,latitude,longitud,design_coor
0,success,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
0,success,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
1,success,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,..."
2,success,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
3,success,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ..."
...,...,...,...,...
3769,success,47.631518,-122.129504,"{'type': 'Point', 'coordinates': [-122.1295044..."
3770,success,47.631518,-122.129504,"{'type': 'Point', 'coordinates': [-122.1295044..."
3774,success,40.756054,-73.986951,"{'type': 'Point', 'coordinates': [-73.986951, ..."
3779,success,59.617364,16.559539,"{'type': 'Point', 'coordinates': [16.5595393, ..."


In [16]:
offices_coor_name.to_json('output/design.json',orient="records")

### Success startup

In [ ]:
query={"$where":[{"category_code":"design"},{"category_code":"web"}]}

offices=list(db.companies.find(query,{"name":1,"offices":1,"_id":0}))
offices=pd.DataFrame(offices)
offices=offices.explode("offices")